# Node connectivity information
This example demonstrates some functionality related to node connectivity, such as:
- Counting number of links connected to a node
- Coutning number of enabled links connected to a node
- Getting a list of upstream/downstream links connected to a node

In [ ]:
# Imports
import pandas as pd
from mikeplus import DataTableAccess
from mikeplus.fieldTableNames.tableNames import CSTabNames

In [ ]:
# Open the database
dta = DataTableAccess("../tests/testdata/Db/Sirius/Sirius.sqlite")
dta.open_database()
dta

In [ ]:
# Get the MsmNodeTable object
msm_node = dta._datatables[CSTabNames.NodeTable].__implementation__
msm_node

In [ ]:
# Get all MUIDs
muids = msm_node.GetMuids()
list(muids)

In [ ]:
# Get number of links connected to Node_36
msm_node.GetNumberofLinks("Node_36")

In [ ]:
# Get number of links connected to Node_36 that are enabled
msm_node.GetNumOfLinksEnabled("Node_36")

In [ ]:
# Get upstream link MUIDs connected to Node_36
# Note: upstream links refers to links where "Node_36" is the upstream node. So in practice, it includes links upstream of "Node_36".
upstream_links = msm_node.GetUpstreamLinks("Node_36")
list(upstream_links)

In [ ]:
# Get downstream link MUIDs connected to Node_36
# Note: downstream links refers to links where "Node_36" is the downstream node. So in practice, it includes links downstream of "Node_36".
downstream_links = msm_node.GetDownstreamLinks("Node_36")
list(downstream_links)

In [ ]:
# Same as above, but for all nodes and visualized in a DataFrame
data = {muid: {
    'num_of_links' : msm_node.GetNumberofLinks(muid),
    'num_of_links_enabled' : msm_node.GetNumberofLinks(muid),
    'num_upstream_links' : msm_node.GetUpstreamLinks(muid).Count,
    'num_downstream_links' : msm_node.GetDownstreamLinks(muid).Count,
} for muid in msm_node.GetMuids()}
df = pd.DataFrame(data).T
df

In [ ]:
# Close the database
dta.close_database()